# 2.4 - Macrobond Data API for Python - Search with Long Results

*Retrieving large list of time series based on search parameters*

This notebook aims to provide examples of how to use Macrobond Data API for Python as well as insights on the key attributes used to display the output in an understandable format.

We will focus on using the Search method based on a few inputs e.g. **Source, Frequency, Category and Region**. This helps you download large list of time series when you start building your universe. 
The emphasis will be on function **'entity_search_multi_filter_long' to pull up to 240,000 time series in the response instead of up to 12,000 using search_entites or entity_search_multi_filter**. In case your response should pull back more than 240,000 entities, we recommend to further narrow down the search using attributes such as Region, Frequency or Category.

You can find a full description of all methods and parameters used in the examples in the [documentation of the API](https://macrobond.github.io/macrobond-data-api/common/api.html).

*Full error handling is omitted for brevity*

***

## Importing packages

In [0]:
import macrobond_data_api as mda
from macrobond_data_api.web import WebClient
from macrobond_data_api.common.types import SearchFilter

import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import time

***

## Searching time series with wide search parameters

Typical Search requests with metadata being fetched can download a maximum of 12,000 entities. With the `entity_search_multi_filter_long()` function, you can retrieve up to 240,000 entities in a single request.

Note that there is no pagination in the response and if your response contains more than 240,000 entities, we recommend you narrow down the search parameters further.

In the example below, we will download the list of time series codes coming from source: "Tourism Research Australia". The code of the data source used in this request is: `src_autra`.

Feel free to use other search parameters. You can find examples in notebooks 2.2 - Filter on a Concept and 2.3 - Filter with a Keyword.

In [0]:
mda.get_one_entity("src_autra")

In [0]:
with WebClient() as api:
    ids = api.entity_search_multi_filter_long(
        SearchFilter(entity_types="TimeSeries", must_have_values={"source": "src_autra"}),
        include_discontinued=False,
    ).entities

print("Number of time series in universe:", len(ids))
ids[:10]

Let's now download the list of time series codes coming from this source.

***

## Fetching the data from the Search

Now that we have generated our universe, we are going to fetch the date, value and metadata on each of these time series.

As industry standard, we recommend our users to download our data by chunks (200 entities each chunk will be ideal) when the universe is large, using `get_entites` and `get_series` methods. 

Alternatively, we also provide convenient solution: our method `get_many_series` can deal with large universe and download the data in one single request, since `get_many_series` handles batches by itself.

In [0]:
df = pd.DataFrame()

payload_ids = [(id, None) for id in ids]

for x in mda.get_many_series(*payload_ids):
    df = pd.concat([df, pd.json_normalize([x.to_dict()])])

df2 = pd.DataFrame(
    df,
    columns=[
        "metadata.Name",
        "metadata.LastModifiedTimeStamp",
        "metadata.StartDate",
        "metadata.LastValueDate",
        "metadata.Frequency",
        "metadata.FullDescription",
    ],
)
df2